In [16]:
# pip install pycountry
import pycountry
# pycountry.countries
# pycountry.languages
import re
from pathlib import Path
import pandas as pd

data_folder = Path("C:/Users/Jing Li/Desktop/cv2db/project/Inputs/intermediate")
file_name = data_folder / "A_Phase1.pkl"
df = pd.read_pickle(file_name)

In [17]:
#ETC

# further seperate
list_edu = list(range(17)) + [53, 54]
list_etc = list(range(17, 53)) + [55]
ETCs = [re.split(r"(?<=\, \d{4})[\s;]", c) if i in list_edu \
        else re.split(r"\n\? ", c) \
        for i, c in enumerate(df['ETC'])]

ETCs_1 = []

for i in ETCs: #each i is a consultant
    temp = list(filter((", and Certifications".__ne__), i))
    temp = list(filter((", and Selected Certifications".__ne__), temp))
    temp = list(filter(("\n".__ne__), temp))
    temp = list(filter((''.__ne__), temp))
    for j in range(len(temp)):
        temp[j] = temp[j].strip()
    ETCs_1.append(temp)

    
# Special case debugging
ETCs_1[3][3:4] = ETCs_1[3][3].split("\n")
ETCs_1[10][1:2] = re.split(r"(?= M.S.,)", ETCs_1[10][1])
ETCs_1[23][2] = re.sub("\nSelected", "", str(ETCs_1[23][2]))

# ETCs_1[30] 
temp = [ETCs_1[30][0]] + ETCs_1[30][1].split("\n* ")
temp[-1] += ' Dentistry'
ETCs_1[30] = [re.sub("[\n\t]", " ", i) for i in temp]

ETCs_1[46][0] += " " + ETCs_1[46][1]
del ETCs_1[46][1]

# special = []
# for i in [k for k in range(17,56) if k not in special]:
#     clean_ETC(i)
    

In [18]:
# seperate ETC into education and training/certificate
edu = []
tr_cer = []
pattern_edu = r"University|Master of|School|College|DrPH|Bachelor|MPH"

certi_list = ['Temple University, Secondary Education Teaching Certification, Philadelphia Pennsylvania, 1977',
              'Certificate in Biostatistics (Master’s level), Georgetown University, 2016',
              'Washington Hospital Center, Georgetown University Hospital, Fellow in Infectious Diseases, Washington DC, 2003 – 2005']

for i in ETCs_1: # for every consultant
    edu_temp = []
    tr_cer_temp = []
    for j in i:
#         print(j, re.match(pattern_edu, j))
        j = re.sub("[•\t]", "", j)
        j = j.strip()
        if re.search(pattern_edu, j) and j not in certi_list :
            edu_temp.append(j)
        else:
            tr_cer_temp.append(j)
    edu.append(edu_temp)
    tr_cer.append(tr_cer_temp)

In [19]:
#Extract Certifications/Licenses

certificates = {"PMP: Project Management Professional": ["Project Management Professional"],
                "CPC: Certified Professional Coach": ["Professional Certified Coach", " Certified Professional Coach"],
                "Residency": ["Residency"],
                "CPA: Certified Public Accountant": ["Certified Public Accountant"],
                "ACCA: Association of Chartered Certified Accountants": ["ACCA"] 
               }

def CheckCertificate(etc):
    p_certificate = []
    for i in certificates:
        for title in certificates[i]:
            if title in etc:
                p_certificate.append(i)
                break
    return p_certificate

Certificates = [CheckCertificate(p) for p in df["ETC"]]

In [20]:
#Extract the highest degree
degrees = {"Ph.D.": ["Postdoctoral", "PhD", "Ph.D.", "Doctor of Philosophy", "Doctoral", "Doctor of Education", "Ed.D.",\
                     "D.D.S.", "DDS", "Doctor of Literature"],
           "M.D.": ["MD", "M.D.", "Doctor of Medicine", "Doctorate in Medicine"],
           "MBA": ["MBA"],
           "MPH": ["MPH", "M.P.H.", "Master of Public Health"],
           "Master": ["Master", "M.S.", "M.S.N.", "M.A.", "M.P.", "Graduate Studies"],
           "Bachelor": ["Bachelor", "BA", "B.A.","A.B.", "B.S.", "BS", "Undergraduate", "MBChB"]}

def CheckDegree (p_edu):
    p_degree = []
    for i in degrees:
        for title in degrees[i]:
            if title in p_edu:
                p_degree.append(i)
                break
    return p_degree

Degrees = [CheckDegree(p) for p in df["ETC"]]

In [21]:

# seperate ETC into education and training/certificate
# edu = []
# tr_cer = []
# pattern_edu = r"University|Master|School|College|DrPH|Bachelor"

# for i in ETCs_1: # for every consultant
#     edu_temp = []
#     tr_cer_temp = []
#     for j in i:
# #         print(j, re.match(pattern_edu, j))
#         if re.search(pattern_edu, j):
#             edu_temp.append(j)
#         else:
#             tr_cer_temp.append(j)
#     edu.append(edu_temp)
#     tr_cer.append(tr_cer_temp)


In [22]:
#Language

ppls_noname = list(df['ppls_data'])

# now an explicit list of languages is used for speed, but packages pycountry can be used in the future
lang_dic = ['Amharic', 'Arabic', 'Bemba', 'Chinese', 'Creole', 'Danish', 'French', 'German', 'Hindi',\
            'Kinyarwanda', 'Kirundi', 'Kiswahili', 'Luo', 'Malayalam', 'Nyanja', 'Portuguese', 'Russian',\
            'Setswana', 'Spanish', 'Swahili', 'Tagalog', 'Tigrigna', 'Tigrinya', 'Yoruba']

lang = []

for i in ppls_noname:
    temp = [j for j in lang_dic if re.search(j, i)]
    lang.append(temp)

In [23]:
#Countries worked/visited/studied
cnty = []
for i in ppls_noname:
    temp = []
    for country in pycountry.countries:
        if re.search(country.name, i):
            temp.append(country.name)
    cnty.append(temp)

In [24]:
#Underserved population worked with

undsrvd_dic = ['HIV', 'homeless', 'low income', 'poverty', 'poor', 'LGBT', 'transgender', 'trafficking',\
           'infants', 'children', 'youth', 'elder', 'disabilit', 'rural', 'low literacy']

#8/12/19/40/41/49/56 disabilities

undsrvd_pop = []
for i in ppls_noname:
    temp = [j for j in undsrvd_dic if re.search(j, i, re.IGNORECASE)]
    temp = ['disabilities' if t == "disabilit" else t for t in temp]
    temp = ['low income' if t == 'poor' else t for t in temp]
    temp = ['low income' if t == 'poverty' else t for t in temp]
    temp = list(set(temp))
    undsrvd_pop.append(temp)
# undsrvd_pop

In [25]:
#Remove ETC, add Education and training/certificate columns
# df["Education"] = edu
# df["Training/Certificate"] = tr_cer
# del df["ETC"]

#Add Edu, Training/Cer, Cert, Deg, languages, countries worked, underserved population
df['ETC'] = ETCs_1
df['Important Certificates'] = Certificates
df["Degrees"] = Degrees
df["Languages"] = lang
df["Countries worked/studied"] = cnty
df["Underserved population worked with"] = undsrvd_pop

In [26]:
# Name Cleaning

for i in [9, 20, 25, 53]:
    df["Name"][i] = re.sub(r"\(\w+\)\s", "", str(df["Name"][i]))

# df["Name"]

In [27]:
#Title Cleaning

for i in [5, 11, 12, 35, 37, 38]:
    df["Title"][i] = re.sub(r"\s\/\s", r"/", str(df["Title"][i]))
df["Title"][49] = re.sub(r"\s\([\w\s]+\)", "", str(df["Title"][49]))

for i in [53, 54]:
    df["Title"][i] = df["Title"][i].strip()
# df["Title"]

In [28]:
#Qualification and Experience Cleaning
for i, w in enumerate(df["Qualification and Experience"]):
    df["Qualification and Experience"][i] = w.strip()

In [29]:
#Career Summary Cleaning

for i, w in enumerate(df['Career Summary']):
    temp = re.sub(r"\n", " ", w)
    df['Career Summary'][i] = re.sub(r"Career Summary", "", temp).strip()

In [30]:
# Storing

file_name = data_folder / "A_Phase2.pkl"
df.to_pickle(file_name)